In [1]:
import os
import dotenv
dotenv.load_dotenv()

True

In [ ]:
## If you want to use OpenAI, uncomment the following lines: 
from openai import OpenAI
config = dotenv_values('.env')
api_key = config.get("API_KEY")
client = OpenAI(api_key=api_key)

In [ ]:
## IF YOU WANT TO USE AZURE OPENAI, UNCOMMENT THE FOLLOWING LINES
# from openai import AzureOpenAI, AsyncAzureOpenAI
# import asyncio

# azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-03-01-preview")
# azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")

# client = AsyncAzureOpenAI(
#     azure_endpoint=azure_openai_endpoint,
#     api_version=azure_openai_api_version,
#     api_key=azure_openai_api_key,
#     azure_deployment="gpt-4o"
# )

In [11]:
async def test_openai_api_key():
    """ 
    Tests whether the environment variable is valid by sending a simple request. 
    Returns True if the key works, False otherwise. 
    """

    try:
        response = await client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {
                    "role": "user",
                    "content": "Write a one-sentence bedtime story about a security breach."
                }
            ])

        print("✅ API key is working.")
        print("Response:", response.choices[0].message.content)

    except Exception as e:
        print(f"⚠️ An error occurred: {str(e)}")
        return False


await test_openai_api_key()

✅ API key is working.
Response: Once upon a time, a clever little firewall befriended a mischievous hacker and, with a gentle nudge, guided him toward using his talents for good, ensuring the kingdom's data was safe and sound forevermore.


# Security Alert Triage Agentic Team

In [ ]:
from pydantic import BaseModel
from agents import Agent, Runner, trace, set_default_openai_client

set_default_openai_client(client)

## Agent to generate or simulate a security alert

In [6]:
alert_generator = Agent(
    name="alert_generator",
    instructions="Generate a simulated security alert based on an environment description."
)


## Agent to assess alert

In [7]:
class AlertAssessment(BaseModel):
    is_legit: bool
    severity: str

triage_agent = Agent(
    name="triage_agent",
    instructions="Evaluate the alert. Return whether it's a legitimate threat and assign severity.",
    output_type=AlertAssessment
)

## Agent to decide escalation

In [8]:
escalation_agent = Agent(
    name="escalation_agent",
    instructions="Based on the alert assessment, decide whether to escalate and provide justification.",
    output_type=str
)

## Run the agents

In [13]:
async def main():
    env_input = input("Describe situation (e.g. failed logins after hours): ")

    with trace("Security Alert Deterministic Flow"):
        # 1. Generate or fetch simulated alert
        alert_result = await Runner.run(alert_generator, env_input)
        print("🔔 Alert Generated: ", alert_result.final_output)

        # 2. Triage it
        triage_result = await Runner.run(triage_agent, alert_result.final_output)
        assessment = triage_result.final_output
        print("🧩 Triage Result: ", assessment)

        # 3. Gate: Only escalate if legit and high severity
        severity = assessment.severity.strip().casefold()
        if not assessment.is_legit:
            print("✔️ Alert is a false positive — stopping here.")
            return
        valid = {"low", "medium", "high"}
        if severity not in valid:
            print(f"⚠️ Unknown severity '{assessment.severity}'. Defaulting to 'medium'.")
            severity = "medium"

        if severity != "high":
            print(f"⚠️ Severity = '{severity}'. Not high — no escalation.")
            return

        # 4. Escalate
        escalation = await Runner.run(escalation_agent, alert_result.final_output)
        print("🚨 Escalation Decision:", escalation.final_output) 

await main()


🔔 Alert Generated:  ---

**Security Alert Notification**

**Date:** [Current Date and Time]  
**Severity Level:** High  
**Category:** Unauthorized Access Attempt

---

**Environment Description:**  
- **Location:** Corporate Headquarters, Main Office Network  
- **Systems Affected:** Employee Workstations, Central Authentication Server  
- **Time Frame:** After business hours (7:00 PM - 6:00 AM)

**Description of Incident:**  
Multiple failed login attempts have been detected on the network after business hours. The attempts were concentrated on several employee workstations and involved incorrect password entries in rapid succession.

**Incident Details:**

1. **Username(s) Targeted:**  
   - User1  
   - User2  
   - User3

2. **Method of Detection:**  
   - Automated monitoring systems identified a pattern of failed logins and triggered an alert.

3. **Number of Failed Logins:**  
   - User1: 15 failed attempts  
   - User2: 10 failed attempts  
   - User3: 20 failed attempts

4. *